In [1]:
from gurobipy import GRB, Model, quicksum
import gurobipy as gb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

In [16]:
food_categories = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/food_categories.csv')
food_preferences = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/food_preferences.csv')
non_profits = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/non_profits.csv')
nurse_shifts = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/nurse_shift_costs.csv')
nutrient_contents = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/nutrient_content.csv')
nutrient_requirements = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/nutrient_requirements.csv')
sp500 = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/sp500_data.csv')
routes = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/Routes.csv')
sunnyshore_financials = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/sunnyshore_financials.csv')
sunnyshore_interest = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/sunnyshore_interest_rates.csv')
vaccines = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/transportation_costs.csv')

# QUESTION 1 - VACCINES

In [17]:
vaccines

,Airport,1-5,6-10,11-15,16-20,21-25,26-29
0,Billy Bishop Toronto City Airport,0.05,0.06,0.07,0.08,0.09,0.10
1,Toronto Pearson Airport,0.08,0.05,0.09,0.10,0.07,0.06


# QUESTION 2 - SP500

In [7]:
sp500

,Ticker symbol,Company,GICS Sector,Location of Headquarters,Price,PercentReturn
0,AEE,Ameren Corp,Utilities,"St. Louis, Missouri",31.26,4.83
1,AXP,American Express Co,Financials,"New York, New York",49.23,1.53
2,T,AT&T Inc,Telecommunications Services,"Dallas, Texas",30.09,5.82
3,AVP,Avon Products,Consumer Staples,"New York, New York",18.31,5.26
4,BFB,Brown-Forman Corporation,Consumer Staples,"Louisville, Kentucky",82.28,1.74
...,...,...,...,...,...,...
62,X,United States Steel Corp.,Materials,"Pittsburgh, Pennsylvania",28.96,0.76
63,UTX,United Technologies,Industrials,"Hartford, Connecticut",77.78,2.63
64,VZ,Verizon Communications,Telecommunications Services,"New York, New York",37.79,4.99
65,WMT,Wal-Mart Stores,Consumer Staples,"Bentonville, Arkansas",61.39,2.44


# QUESTION 3 - ROUTES

In [8]:
routes

,Routes,Cost
0,"[U, a, b, f, e, d, c, U]",136
1,"[U, a, b, f, e, d, U]",110
2,"[U, a, b, f, e, U]",90
3,"[U, a, b, f, d, c, U]",132
4,"[U, a, b, f, d, e, U]",118
5,"[U, a, b, f, d, U]",106
6,"[U, a, c, d, e, f, b, U]",136
7,"[U, a, c, d, e, U]",92
8,"[U, a, c, d, f, b, U]",132
9,"[U, a, c, d, f, e, U]",112


# QUESTION 4 - FOOD CATEGORIES

In [10]:
food_categories

,Food_Item,Is_Vegetarian,Is_Vegan,Is_Kosher,Is_Halal,Cost_per_gram
0,Food_1,1,1,1,1,0.132322
1,Food_2,0,0,0,0,0.157278
2,Food_3,1,1,1,1,0.140415
3,Food_4,0,0,0,0,0.131732
4,Food_5,0,0,1,1,0.113548
...,...,...,...,...,...,...
115,Food_116,0,0,0,0,0.109476
116,Food_117,0,0,1,1,0.182165
117,Food_118,0,0,0,0,0.137191
118,Food_119,1,1,0,0,0.182260


In [15]:
food_preferences

,Veggie_grams,Vegan_grams,Kosher_grams,Halal_grams,All_grams
0,46160.0,11540.0,17310.0,92320.0,577000


In [13]:
nutrient_contents

,Unnamed: 0,Nutrient_1,Nutrient_2,Nutrient_3,Nutrient_4,Nutrient_5,Nutrient_6,Nutrient_7,Nutrient_8,Nutrient_9,...,Nutrient_51,Nutrient_52,Nutrient_53,Nutrient_54,Nutrient_55,Nutrient_56,Nutrient_57,Nutrient_58,Nutrient_59,Nutrient_60
0,Food_1,0.007527,0.005512,0.009605,0.006796,0.004815,0.006458,0.001173,0.003714,0.006942,...,0.005689,0.001489,0.002800,0.001167,0.008143,0.003015,0.004108,0.009353,0.007340,0.001287
1,Food_2,0.002482,0.006593,0.006195,0.003141,0.009408,0.006526,0.005821,0.006309,0.007571,...,0.003285,0.002920,0.005664,0.001231,0.002867,0.004822,0.004368,0.005172,0.003499,0.006281
2,Food_3,0.008775,0.002058,0.005656,0.002189,0.007452,0.004565,0.006089,0.002650,0.002304,...,0.004317,0.002232,0.008399,0.002709,0.005602,0.003019,0.001881,0.008760,0.009756,0.009648
3,Food_4,0.009159,0.007966,0.003998,0.001730,0.004665,0.003090,0.002192,0.001481,0.007530,...,0.006810,0.001318,0.004874,0.005590,0.005826,0.007133,0.003498,0.002160,0.004534,0.009608
4,Food_5,0.002684,0.009136,0.005894,0.005112,0.008938,0.005127,0.007518,0.004591,0.009136,...,0.001140,0.004859,0.001613,0.003267,0.002990,0.003279,0.002179,0.001108,0.002039,0.006566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Food_116,0.002451,0.008111,0.009838,0.001967,0.008573,0.005408,0.001463,0.005692,0.005337,...,0.007032,0.004821,0.007372,0.006519,0.004974,0.005941,0.006383,0.008912,0.007092,0.007424
116,Food_117,0.009729,0.001609,0.002689,0.003863,0.009515,0.003057,0.006086,0.002060,0.003436,...,0.006986,0.009281,0.006644,0.008465,0.007158,0.009525,0.004665,0.007955,0.005583,0.001445
117,Food_118,0.001119,0.004826,0.006940,0.008769,0.001294,0.009477,0.008963,0.008863,0.006784,...,0.001702,0.005247,0.006531,0.009452,0.009291,0.006467,0.005524,0.006854,0.009325,0.009918
118,Food_119,0.001298,0.009726,0.008229,0.001675,0.008202,0.006964,0.009152,0.005120,0.005530,...,0.006204,0.001681,0.003823,0.009844,0.006520,0.004778,0.009241,0.004252,0.007724,0.002316


In [14]:
nutrient_requirements

,Nutrient,Min_Requirement,Max_Requirement
0,Nutrient_1,3831.321698,8474.081720
1,Nutrient_2,2251.071301,8403.640809
2,Nutrient_3,2975.778588,9619.196273
3,Nutrient_4,3284.851614,9534.776337
4,Nutrient_5,3139.208223,9622.440982
5,Nutrient_6,3778.988620,9514.680273
6,Nutrient_7,3368.066911,8191.724092
7,Nutrient_8,3650.066262,9765.756832
8,Nutrient_9,3324.251821,8755.235098
9,Nutrient_10,2688.486944,8439.329026


# QUESTION 5 - NON-PROFITS

In [9]:
non_profits

,alpha_i,beta_i
0,54.881350,16.469416
1,71.518937,62.147840
2,60.276338,57.722859
3,54.488318,23.789282
4,42.365480,93.421400
...,...,...
175,22.392469,68.139251
176,34.535168,27.759610
177,92.808129,12.886057
178,70.441440,39.267568


# QUESTION 6 - NURSE SHIFTS

In [11]:
nurse_shifts

,Nurse_ID,Category,Cost_Weekday,Cost_Weekend,Cost_Overtime
0,1,SRN,150,180,248
1,2,RN,360,432,594
2,3,SRN,600,720,990
3,4,RN,120,144,198
4,5,RN,240,288,396
5,6,NIT,320,384,528
6,7,SRN,600,720,990
7,8,NIT,320,384,528
8,9,SRN,150,180,248
9,10,SRN,300,360,495


# QUESTION 7 - SUNNYSHORE BAY

In [18]:
sunnyshore_financials

,Month,Revenues,Expenses
0,May,180000,300000
1,June,260000,400000
2,July,420000,350000
3,August,580000,200000


In [19]:
sunnyshore_interest

,Term,Interest Rate (%)
0,1-month,1.75
1,2-months,2.25
2,3-months,2.75
